## Imports

In [2]:
#!pip install torch
#!pip install xgboost
#!pip install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --user
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#os.environ["WORLD_SIZE"] = "1"
import torch
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torch
import glob
import numpy as np
import pandas as pd
from matplotlib.lines import Line2D
from tqdm import tqdm
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Team1Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(2, 16))
        self.pool = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=(1, 8))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 8))
        #self.fc1 = nn.Linear(in_features=15360, out_features=1200)  # Fully Connected Layer
        self.fc1 = nn.LazyLinear(out_features=1200)
        self.fc2 = nn.Linear(in_features=1200, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=65)
        self.fc4 = nn.Linear(in_features=65, out_features=self.num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # print(f'after conv1: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = F.relu(self.conv2(x))
        # print(f'after conv2: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = F.relu(self.conv3(x))
        # print(f'after conv3: {x.shape}')
        x = self.pool(x)
        # print(f'after pool: {x.shape}')
        x = x.reshape(x.shape[0], -1)
        # print(f'after reshaping: {x.shape}')
        x = F.relu(self.fc1(x))
        # print(f'fc1: {x.shape}')
        x = F.relu(self.fc2(x))
        # print(f'fc2: {x.shape}')
        x = F.relu(self.fc3(x))
        # print(f'fc3: {x.shape}')
        #x = F.relu(self.fc4(x))
        x = self.fc4(x)
        # print(f'fc3: {x.shape}')

        return x

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Team2Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(2, 128, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 256, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(256),
            nn.Conv1d(256, 512, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(512),
            nn.Conv1d(512, 1024, 8, dtype=torch.float32),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.BatchNorm1d(1024),
        )

        self.fc1 = nn.LazyLinear(512, dtype=torch.float32)
        self.fc2 = nn.Linear(512, num_classes, dtype=torch.float32)
        # self.fc = nn.Sequential(
        #     nn.LazyLinear(512, dtype=torch.float32),
        #     nn.ReLU(),
        #     nn.Linear(512, num_classes, dtype=torch.float32),
        #     nn.ReLU(),
        # )

    def forward(self, x):
        x = x.squeeze(1)
        for layer in self.conv_layers:
            x = layer(x)
        x = x.flatten(1)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #return F.softmax(x, dim=1)
        return x

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# From headley_modrec.py
class Team3Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(1, 16, (2, 16))
        self.conv2 = nn.Conv2d(16, 8, (1, 8))
        self.conv3 = nn.Conv2d(8, 4, (1, 4))
        #self.fc1 = nn.Linear(3996, 512)
        self.fc1 = nn.LazyLinear(out_features=512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, self.num_classes)

        self.activation = {}

    def forward(self, x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = F.tanh(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [6]:
class Team4Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.re1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.re2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.re3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.re4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2))

        self.flat = nn.Flatten()
        self.drop1 = nn.Dropout(0.5)
        #self.fc1 = nn.Linear(128 * 256,512)  # I dont exactly know why it is 128x256, but I had to do some debugging and hardcode the required value
        self.fc1 = nn.LazyLinear(out_features=512)

        self.re5 = nn.ReLU()
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.re6 = nn.ReLU()
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool1(self.re1(self.conv1(x)))
        x = self.pool2(self.re2(self.conv2(x)))
        x = self.pool3(self.re3(self.conv3(x)))
        x = self.pool4(self.re4(self.conv4(x)))
        x = self.flat(x)
        x = self.drop1(x)
        x = self.re5(self.fc1(x))
        x = self.drop2(x)
        x = self.re6(self.fc2(x))
        x = self.fc3(x)
        return x

## Loading models and helper functions

In [7]:
sig_types = [['2-ASK', ['ask', 2], 0],
             ['4-ASK', ['ask', 4], 1],
             ['8-ASK', ['ask', 8], 2],
             ['BPSK', ['psk', 2], 3],
             ['QPSK', ['psk', 4], 4],
             ['16-QAM', ['qam', 16], 5],
             ['Tone', ['constant'], 6],
             ['P-FMCW', ['p_fmcw'], 7]]
num_classes = len(sig_types)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

team1_model = Team1Model(num_classes)
team1_model.load_state_dict(torch.load('../data/summer_models/team1_model.pt', map_location=torch.device('cpu')))
team1_model.eval()
team1_model.to(device)
print ("loaded team1")

team2_model = Team2Model(num_classes)
team2_model.load_state_dict(torch.load('../data/summer_models/team2_model.pt', map_location=torch.device('cpu')))
team2_model.eval()
team2_model.to(device)
print ("loaded team2")
team3_model = Team3Model(num_classes)
team3_model.load_state_dict(torch.load('../data/summer_models/team3_model.pt', map_location=torch.device('cpu')))
team3_model.eval()
team3_model.to(device)
print ("loaded team3")
team4_model = Team4Model(num_classes)
team4_model.load_state_dict(torch.load('../data/summer_models/team4_model.pt', map_location=torch.device('cpu')))
team4_model.eval()
team4_model.to(device)
print ("loaded team4")

cpu
loaded team1


/tmp/ipykernel_3623/2531680920.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  team1_model.load_state_dict(torch.load('../data/summer_models/team1_model.pt', map_locati

loaded team2
loaded team3
loaded team4


/tmp/ipykernel_3623/2531680920.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  team3_model.load_state_dict(torch.load('../data/summer_models/team3_model.pt', map_locati

In [8]:
MODELS_OBS_INT = 2048
def load_data(channel_path, batch_size, num_batches, num_train_examples, data_obs_int):
    training_data = np.zeros((num_train_examples, 1, 2, MODELS_OBS_INT), dtype=np.float32)

    last_index = 0
    for k in range(num_batches):
        # This is used if we have a labeldata folder that stores class labels
        label_df = pd.read_csv(f"{channel_path}/labeldata/example_{k + 1}.csv")

        iq_data = np.fromfile(f"{channel_path}/iqdata/example_{k + 1}.dat", np.csingle)
        iq_data = np.reshape(iq_data, (-1, data_obs_int))  # Turn the IQ data into chunks of (chunk size) x (data_obs_int)
        for j in range(iq_data.shape[0]):
            iq_array_norm = iq_data[j][:] / np.max(np.abs(iq_data[j][:]))  # Normalize the observation
            iq_array = np.vstack((iq_array_norm.real, iq_array_norm.imag))  # Separate into 2 subarrays - 1 with only real (in-phase), the other only imaginary (quadrature)

            # Pad the iq array with zeros to meet the observation length requirement
            # This is needed because the CNN models have a fixed input size
            iq_array = np.pad(iq_array, ((0, 0), (0, MODELS_OBS_INT - iq_array[0].size)), mode='constant', constant_values=0)

            training_data[last_index, 0, :, :] = iq_array
            last_index += 1
    return torch.utils.data.DataLoader([training_data[i] for i in range(num_train_examples)], batch_size=batch_size, shuffle=False)


In [9]:
def load_labels(channel_path, batch_size, num_batches, num_train_examples, data_obs_int):
    training_labels = torch.stack([torch.nn.functional.one_hot(torch.tensor(pd.read_csv(os.path.join(channel_path, f'labeldata/example_{i+1}.csv')).iloc[:,0])) for i in range(num_batches)])
    # print (training_labels)
    return training_labels
    # return torch.utils.data.DataLoader(training_labels, batch_size=batch_size, shuffle=False)

In [10]:
num_batches = 21
CHUNK_SIZE = 5000
num_train_examples = num_batches * CHUNK_SIZE
OBS_INT = 2048

labels = load_labels("../data/dataset/team1/train", 21, num_batches, num_train_examples, 2048)
print (type(labels))

print(labels.shape)
count_all_zeros = 0
labels = labels.numpy()
for i in range(len(labels)):
    if np.sum(labels[i]) == 0:
        count_all_zeros += 1

print("number of missing labels:", count_all_zeros)

<class 'torch.Tensor'>
torch.Size([21, 5000, 8])
number of missing labels: 0


In [11]:
#set directory paths
#change to correct for testing data
train_dir1 = os.path.join('../data/dataset', 'team1', 'train')
train_iq_files = os.path.join(train_dir1, 'iqdata', 'example_*.dat')
train_dir2 = os.path.join('../data/dataset', 'team2', 'train')
train_iq_files = os.path.join(train_dir2, 'iqdata', 'example_*.dat')
train_dir3 = os.path.join('../data/dataset', 'team3', 'train')
train_iq_files = os.path.join(train_dir3, 'iqdata', 'example_*.dat')
train_dir4 = os.path.join('../data/dataset', 'team4', 'train')
train_iq_files = os.path.join(train_dir4, 'iqdata', 'example_*.dat')

num_batches = 21
CHUNK_SIZE = 5000
num_train_examples = num_batches * CHUNK_SIZE
OBS_INT = 2048
team1_train_dataloader = load_data(train_dir1, 1, num_batches, num_train_examples, 2048)
team2_train_dataloader = load_data(train_dir2, 1, num_batches, num_train_examples, 1024)
team3_train_dataloader = load_data(train_dir3, 1, num_batches, num_train_examples, 512)
team4_train_dataloader = load_data(train_dir4, 1, num_batches, num_train_examples, 256)
print ("loaded data")


loaded data


## Feature extraction

In [12]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

In [13]:
# team 1 features
features = {}

selected_layer = getattr(team1_model, "fc3")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))
print(selected_layer)

preds_list = []
feats_list1 = []


# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team1_train_dataloader)):
    with torch.inference_mode():
        preds = team1_model(inputs.to(device))
    feats_list1.append(features['feats'].cpu().numpy())

feats_list1 = np.concatenate(feats_list1)

features1 = np.array(feats_list1)
features1 = torch.tensor(features1)
features1 = features1.reshape(-1, features1.shape[-1])
print("features size: ", features1.size())
print (type(features1))

Linear(in_features=100, out_features=65, bias=True)


105000it [02:02, 856.15it/s]


features size:  torch.Size([105000, 65])
<class 'torch.Tensor'>


In [14]:
# team 2 features
features = {}

selected_layer = getattr(team2_model, "fc1")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))
print(selected_layer)

preds_list = []
feats_list2 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team2_train_dataloader)):
    with torch.inference_mode():
        preds = team2_model(inputs.to(device))
    feats_list2.append(features['feats'].cpu().numpy())

feats_list2 = np.concatenate(feats_list2)

features2 = np.array(feats_list2)
features2 = torch.tensor(features2)
features2 = features2.reshape(-1, features2.shape[-1])
print("features size: ", features2.size())
print (type(features2))

LazyLinear(in_features=0, out_features=512, bias=True)


105000it [03:11, 548.57it/s]


features size:  torch.Size([105000, 512])
<class 'torch.Tensor'>


In [15]:
# team 3 features
features = {}

selected_layer = getattr(team3_model, "fc3")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))
print(selected_layer)

preds_list = []
feats_list3 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team3_train_dataloader)):
    with torch.inference_mode():
        preds = team3_model(inputs.to(device))
    feats_list3.append(features['feats'].cpu().numpy())

feats_list3 = np.concatenate(feats_list3)

features3 = np.array(feats_list3)
features3 = torch.tensor(features3)
features3 = features3.reshape(-1, features3.shape[-1])
print("features size: ", features3.size())
print (type(features3))

Linear(in_features=128, out_features=64, bias=True)


105000it [24:20, 71.89it/s] 


features size:  torch.Size([105000, 64])
<class 'torch.Tensor'>


In [16]:
# team 4 features
features = {}

selected_layer = getattr(team4_model, "fc2")  #sometimes its just model or model.module
input_features = selected_layer.in_features
selected_layer.register_forward_hook(get_features('feats'))
print(selected_layer)

preds_list = []
feats_list4 = []

# Feed the IQ data into the model
for idx, inputs in tqdm(enumerate(team4_train_dataloader)):
    with torch.inference_mode():
        preds = team4_model(inputs.to(device))
    feats_list4.append(features['feats'].cpu().numpy())

feats_list4 = np.concatenate(feats_list4)

features4 = np.array(feats_list4)
features4 = torch.tensor(features4)
features4 = features4.reshape(-1, features4.shape[-1])
print("features size: ", features4.size())
print (type(features4))

Linear(in_features=512, out_features=256, bias=True)


105000it [24:33, 71.26it/s] 


features size:  torch.Size([105000, 256])
<class 'torch.Tensor'>


In [17]:
combined_tensor = torch.cat((features1, features2, features3, features4), dim=1)
print(f"Combined_tensor size: {combined_tensor.size()}")

Combined_tensor size: torch.Size([105000, 897])


In [18]:
labels_list = []

testloader = load_labels("../data/dataset/team1/train", 1, num_batches, num_train_examples, 2048)
x = 0
for batch in testloader:
    #print(batch)
    if torch.sum(batch) == 0:
        print(f'Found all zeros at index {x}')
    labels = batch  
    labels_list.append(labels)
    x += 1

all_labels = torch.cat(labels_list, dim=0)
labels= all_labels.numpy()
print("labels shape: ", labels.shape)
# print (labels.shape)
#torch.save(labels, "new_updated_labels_second_to_last.pt")
# print (combined_tensor.shape)
# print(labels.reshape(105000, 8)[75000:80000])

labels shape:  (105000, 8)


## Training model/saving extracted features

In [19]:
# Save the extracted features and labels for fusion in other scripts
fusion_data_folder = '../data/fusion_data'

# Save features for the individual models
with open(os.path.join(fusion_data_folder, 'team1_features_second_to_last.npy'), 'wb') as f:
    np.save(f, features1.numpy())
with open(os.path.join(fusion_data_folder, 'team2_features_second_to_last.npy'), 'wb') as f:
    np.save(f, features2.numpy())
with open(os.path.join(fusion_data_folder, 'team3_features_second_to_last.npy'), 'wb') as f:
    np.save(f, features3.numpy())
with open(os.path.join(fusion_data_folder, 'team4_features_second_to_last.npy'), 'wb') as f:
    np.save(f, features4.numpy())

# Save combined features
with open(os.path.join(fusion_data_folder, 'combined_features_allsecond_to_last.npy'), 'wb') as f:
    np.save(f, combined_tensor.numpy())

# Save labels
with open(os.path.join(fusion_data_folder, 'labels.npy'), 'wb') as f:
    np.save(f, labels)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(combined_tensor.numpy(), labels.reshape(105000, 8), test_size=0.2, random_state=42)
params = {
    'objective': 'multi:softmax',  
    'num_class': num_classes,      
    'eval_metric': 'merror',       
    'max_depth': 6,                
    'min_child_weight': 1,         
    'subsample': 0.8,              
    'colsample_bytree': 0.8,       
    'learning_rate': 0.1,          
    'n_estimators': 100            
}

num_rounds = 15  
print ("split")

#print(len(X_train))
#print(len(X_test))
print(X_train)
print(y_train)

model_fused = xgb.XGBClassifier(tree_method="hist")
model_fused.fit(X_train, y_train, eval_set=[(X_test, y_test)])

preds = model_fused.predict(X_test).argmax(1)
#preds = model_fused.predict(X_test)
print(preds)
acc = (preds == y_test.argmax(1)).sum() / preds.shape[0]
print(f"Accuracy: {acc}")

split
[[-9.5405815e+01 -1.0237858e+02 -4.9376961e+01 ... -7.5925851e+00
  -8.4549570e+00 -6.4325814e+00]
 [-3.4990044e+01 -8.1022453e+01 -1.6680246e+01 ... -7.8388405e-01
  -2.3498110e+01 -2.1892767e+01]
 [-2.8673544e+02  1.2923242e+02  2.1418201e+03 ... -9.5032568e+00
  -1.3903454e+01 -1.2622296e+01]
 ...
 [-6.0012109e+02 -5.6118690e+01  4.5613013e+03 ... -9.0246153e+00
  -1.2385365e+01 -1.0883750e+01]
 [-1.5780228e+01 -2.2764688e+01 -1.3417933e+01 ... -1.6095691e+00
  -8.7466068e+00 -6.1196389e+00]
 [-3.7305004e+01 -7.1418030e+01 -6.3897972e+01 ... -1.2154825e+01
  -9.3149185e+00 -1.6690149e+01]]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0]	validation_0-logloss:0.20531
[1]	validation_0-logloss:0.14276
[2]	validation_0-logloss:0.10217
[3]	validation_0-logloss:0.07427
[4]	validation_0-logloss:0.05457
[5]	validation_0-logloss:0.04041
[6]	validation_0-logloss:0.03013
[7]	validation_0-logloss:0.02265
[8]	valida

In [21]:
model_fused.save_model('../data/summer_models/baseline_fused_2tl.json')